# Experiment 1
This notebook contains the code for the first experiment of the paper. In this experiment, we train a B2BNet model that first encodes the temporal dimensions of the EEG data and then its spatial dimensions. It does not include any B2B or classification heads.

In [ ]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
# Parameters
segment_size = 120 * 3 # 3sec
batch_size = 256
n_channels = 59
space_embedding_dim = 7
time_embedding_dim = 16
max_epochs = 1000

In [ ]:
datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size, data_mode='crop')

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, space_embedding_dim=space_embedding_dim,
    time_embedding_dim=time_embedding_dim, kernel_size=1, n_subjects=52)

trainer = pl.Trainer(max_epochs=max_epochs, accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)